## Import

In [2]:
import pandas as pd
import numpy as np
from sklearn.metrics.pairwise import cosine_similarity

In [4]:
import csv



mydict = {}
with open("apply_train.csv", "r") as file:
    reader = csv.reader(file)
    attrs = []
    result = []
    for row in reader:
        if len(attrs) < 1:
            attrs = row
            continue
        temp_row_dict = dict()
        for key, value in zip(attrs, row):
            temp_row_dict[key] = value
        result.append(temp_row_dict)


for row in result:
    print(row)

FileNotFoundError: [Errno 2] No such file or directory: 'apply_train.csv'

## Data Load

In [1]:
apply_train_df = pd.read_csv('apply_train.csv')
apply_train_df

NameError: name 'pd' is not defined

In [16]:
my_2darray = np.array([[1, 2, 3], [4, 5, 6]])
display(pd.DataFrame(my_2darray))

# Take a dictionary as input to your DataFrame 
my_dict = {"a": ['1', '3'], "b": ['1', '2'], "c": ['2', '4']}
display(pd.DataFrame(my_dict))

,0,1,2
0,1,2,3
1,4,5,6


,a,b,c
0,1,1,2
1,3,2,4


## User-Item Matrix / Similarity / Score

In [13]:
# 사용자-아이템 행렬 생성: 구직자가 해당 채용 공고에 지원했으면 1, 아니면 0으로 설정
user_item_matrix = apply_train_df.groupby(['resume_seq', 'recruitment_seq']).size().unstack(fill_value=0)
user_item_matrix[user_item_matrix > 1] = 1

print(user_item_matrix.shape)
# 사용자 간의 유사성 계산
user_similarity = cosine_similarity(user_item_matrix)

# 추천 점수 계산
user_predicted_scores = user_similarity.dot(user_item_matrix) / np.array([np.abs(user_similarity).sum(axis=1)]).T

(8482, 6695)


## Prediction

In [4]:
# 이미 지원한 채용 공고 제외하고 추천
recommendations = []
for idx, user in enumerate(user_item_matrix.index):
    # 해당 사용자가 지원한 채용 공고
    applied_jobs = set(user_item_matrix.loc[user][user_item_matrix.loc[user] == 1].index)
    
    # 해당 사용자의 추천 점수 (높은 점수부터 정렬)
    sorted_job_indices = user_predicted_scores[idx].argsort()[::-1]
    recommended_jobs = [job for job in user_item_matrix.columns[sorted_job_indices] if job not in applied_jobs][:5]
    
    for job in recommended_jobs:
        recommendations.append([user, job])

## Submission

In [5]:
    # sample_submission.csv 형태로 DataFrame 생성
top_recommendations = pd.DataFrame(recommendations, columns=['resume_seq', 'recruitment_seq'])

top_recommendations.to_csv('./baseline_submit.csv', index=False)